In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from selenium import webdriver
import requests
import os

In [2]:
os.chdir('D:\\Scripting')

In [3]:
url = 'https://devtomanager.com/interviews/'

In [4]:
browser = webdriver.Firefox()

In [5]:
browser.get(url)

In [6]:
html = browser.page_source

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
soup.find_all('h5', class_ = 'card-title')[0].text.strip().split(',')

['Aviv Ben-Yosef', ' Tech Executive Consultant']

In [9]:
### Fetching employee name
employee_name = [i.text.strip().split(',')[0] for i in soup.find_all('h5', class_ = 'card-title')]
employee_name

['Aviv Ben-Yosef', 'Anand Safi', 'Shawn Axsom', 'Kevin Doyle', 'Arnab Sen']

In [10]:
### Fetching Job Position
job_pos = [i.text.strip().split(',')[1].split(' at ')[0].strip() for i in soup.find_all('h5', class_ = 'card-title')]
job_pos

['Tech Executive Consultant',
 'Engineering Manager',
 'Senior Engineering Manager',
 'CTO',
 'Technical Delivery Manager']

In [11]:
### Fetching working at. if working is not present then substituted it with np.nan
import numpy as np
working_at = [i.text.strip().split(',')[1].split(' at ')[-1].strip() if ' at ' in i.text.strip() else np.nan for i in soup.find_all('h5', class_ = 'card-title')]
working_at

[nan, 'Mark43', 'Docker', 'patientMpower', 'AKQA']

- As employee no 1 working is not mentioned so substituded it with np.nan
- So it will be blank space in the working of first employee

In [12]:
### Extracting interview quote
Interview_quote =[i.text.strip() for index,i in enumerate(soup.select('p.card-text')) if index%2==0]
Interview_quote

['“I want to set up managers to succeed, no matter how much background they’ve already got.”',
 '“I very well understand where my interests lie, which is being an enabler first and then a creator rather than being a creator/ maker 100%.”',
 '“Your focus (and challenges) grows from your direct reports, to teams, to departments, executives, and then external customers and partnerships.”',
 '“If I can provide enough direction to allow people to focus on the jobs they’ll do better than I ever could, everybody wins.”',
 '“The switch to management can be highly rewarding and provide a level of independence, authority, and interactivity that a pure software engineering role may not offer.”']

In [13]:
Date = [t for index,t in enumerate([i.text.split('\n')[1].strip() for i in soup.find_all('p', class_ = 'card-text')]) if index%2==1]
Date

['August 16, 2021',
 'July 19, 2021',
 'July 05, 2021',
 'June 21, 2021',
 'May 10, 2021']

In [14]:
Tags = [t for index,t in enumerate([[g.text.strip() for g in t] for t in [i.select('a.text-reset') for i in soup.select('p.card-text')]]) if index%2==1]
Tags

[['#coaching', '#consulting'],
 ['#coaching', '#frontend', '#full-stack', '#public-safety'],
 ['#backend', '#coaching', '#information-systems', '#web'],
 ['#full-stack', '#health-tech'],
 ['#consulting', '#digital-marketing', '#e-commerce', '#product']]

In [15]:
df = pd.DataFrame({'Employee Name':employee_name, 'Job Role': job_pos, 'Working At': working_at, 'Quote': Interview_quote, 'Date': Date, 'Tags': Tags})

In [16]:
def dovtomanager(soup):
    employee_name = [i.text.strip().split(',')[0] for i in soup.find_all('h5', class_ = 'card-title')]
    job_pos = [i.text.strip().split(',')[1].split(' at ')[0].strip() for i in soup.find_all('h5', class_ = 'card-title')]
    working_at = [i.text.strip().split(',')[1].split(' at ')[-1].strip() if ' at ' in i.text.strip() else np.nan for i in soup.find_all('h5', class_ = 'card-title')]
    Interview_quote =[i.text.strip() for index,i in enumerate(soup.select('p.card-text')) if index%2==0]
    Date = [t for index,t in enumerate([i.text.split('\n')[1].strip() for i in soup.find_all('p', class_ = 'card-text')]) if index%2==1]
    Tags = [t for index,t in enumerate([[g.text.strip() for g in t] for t in [i.select('a.text-reset') for i in soup.select('p.card-text')]]) if index%2==1]
    df = pd.DataFrame({'Employee Name':employee_name, 'Job Role': job_pos, 'Working At': working_at, 'Quote': Interview_quote, 'Date': Date, 'Tags': Tags})
    df['Date'] = pd.to_datetime(df['Date'])
    return(df)

In [17]:
stud = 'https://devtomanager.com/interviews/page/{}/'

In [18]:
from tqdm.notebook import tqdm
from time import sleep

In [19]:
tables = []
for i in tqdm(range(1, 6)):
    url_pages = stud.format(i)
    browser.get(url_pages)
    sleep(5)
    html = browser.page_source
    soup2 = BeautifulSoup(html, 'html.parser')
    tables.append(dovtomanager(soup2))

  0%|          | 0/5 [00:00<?, ?it/s]

In [20]:
manager = pd.concat(tables, axis = 0, ignore_index = True)
manager

,Employee Name,Job Role,Working At,Quote,Date,Tags
0,Aviv Ben-Yosef,Tech Executive Consultant,NaN,"“I want to set up managers to succeed, no matt...",2021-08-16,"[#coaching, #consulting]"
1,Anand Safi,Engineering Manager,Mark43,“I very well understand where my interests lie...,2021-07-19,"[#coaching, #frontend, #full-stack, #public-sa..."
2,Shawn Axsom,Senior Engineering Manager,Docker,“Your focus (and challenges) grows from your d...,2021-07-05,"[#backend, #coaching, #information-systems, #web]"
3,Kevin Doyle,CTO,patientMpower,“If I can provide enough direction to allow pe...,2021-06-21,"[#full-stack, #health-tech]"
4,Arnab Sen,Technical Delivery Manager,AKQA,“The switch to management can be highly reward...,2021-05-10,"[#consulting, #digital-marketing, #e-commerce,..."
5,Nicholas Cobb,Senior Manager of Software Engineering,Aurora Innovation,“Realizing that life as a manager will have it...,2021-04-26,"[#autonomous-driving, #devops, #ios, #self-tau..."
6,Christoph Eicke,Team Lead,Xing GmbH,“Sometimes it's better not to know the details...,2021-04-12,"[#full-stack, #linux, #self-taught, #web]"
7,Sam Phillips,Co-founder & CTO,Residently,“I think leadership and management are roles t...,2021-03-29,"[#coaching, #devops, #self-taught, #startup, #..."
8,Dan Lines,Co-founder & COO,LinearB,“Some sound advice I received is that if your ...,2021-03-15,"[#cybersecurity, #entrepreneurship, #startup]"
9,Duncan Skelton,Executive Coach,Duncan Skelton Coaching,"“If you get the job, or are new to the role, t...",2021-03-01,"[#coaching, #enterprise, #hardware, #systems-p..."


- Note:  As employee no. 1 is not working at any company Nan is shown which means that that cell is empty.

In [23]:
manager.to_csv(r'C:\Users\Admin\Desktop\Assignment\Data Scraping\Assignmetn solution\Question 3/dovtomanaget Site DataScraping.csv')